# Transforming html to txt

<p>First, we need to extract the text from the pages. For this, we gonna use the Beautiful Soup parser in each page from local directory - the positives and negatives. We extract from paragraphs of html and append the results. Some pages has irrelevant information, like ads - we will try to remove after this.</p>
<p>After append the strings, we append the result (1 or 0), 1 if is a relevant page and 0 otherwise.</p>

In [12]:
import os
import time
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as soup
import re
import string

In [47]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lavinia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [49]:
path = "pages/positivos"

parsedPositive = []
parsed = []
y = []

stopwords = set(stopwords.words('portuguese') + list(punctuation))

for filename in os.listdir(path):
    fullpath = os.path.join(path, filename)
    print(fullpath)
    if fullpath.endswith('.html'):
        page = soup(open(fullpath),'html.parser')
        table = page.find_all('p')
        s = "";
        pala_without_stop = "";
        p = [];
        for x in table:
                if(x.text != 'placeholder'):
                    s += x.text + " ";
        
        s=''.join(i for i in s if  not i.isdigit())
        for i in s.split():
            if i not in stopwords:
                pala_without_stop += i + " ";
        pala_without_stop = pala_without_stop.lower();
        pala_without_stop = pala_without_stop.translate(string.punctuation);
        parsed.append(pala_without_stop);
        y.append(1);

pages/positivos/udon.html
pages/positivos/proteina-de-soja-com-curry.html
pages/positivos/refogado-de-mandioquinha-e-proteina-de-soja.html
pages/positivos/batatas-ao-murro.html
pages/positivos/moqueca-de-palmito.html
pages/positivos/muffin-de-cenoura-integral.html
pages/positivos/charuto-de-repolho-com-recheio-de-proteina-de-soja.html
pages/positivos/Maionese de Cenoura.html
pages/positivos/canape-de-abobrinha.html
pages/positivos/risotto-de-abobora-cabotia.html
pages/positivos/onion-rings.html
pages/positivos/cogumelos-paris-xadrez.html
pages/positivos/refogado-de-brocolis-e-shiitake.html
pages/positivos/macarrao-ao-molho-funghi.html
pages/positivos/cookies-de-okara.html
pages/positivos/maionese-de-castanha-de-caju.html
pages/positivos/Receita de Cogumelo shitake na manteiga - Fácil.html
pages/positivos/hamburguer-de-lentilha.html
pages/positivos/receita-de-antepasto-de-berinjela-agridoce-vapt-vupt-6839.html
pages/positivos/salada-de-bifum.html
pages/positivos/torta-de-liquidificador.

pages/positivos/beijinho-funcional.html
pages/positivos/torta-de-doce-de-leite-de-amendoim-ameixa-e-coco.html
pages/positivos/bolo-vegano-de-caneca.html
pages/positivos/picole-de-limao-e-kiwi.html
pages/positivos/cestinhas-de-massa-de-pastel.html
pages/positivos/como-fazer-pamonha.html
pages/positivos/churrascovegano.html
pages/positivos/Receita de Batatas Carameladas.html
pages/positivos/receita-de-hamburguer-caseiro-vegano-6699.html
pages/positivos/Sopa Indiana de Grão de Bico - gordelícias.htm
pages/positivos/manjar-de-coco.html
pages/positivos/incremente-sua-sopa.html
pages/positivos/tofu-a-parmegiana.html
pages/positivos/torta-aberta-de-legumes.html
pages/positivos/pure-de-batata-com-couve-flor.html
pages/positivos/esfirra-vegana.html
pages/positivos/torta-de-morangos.html
pages/positivos/Couve Flor Assada - gordelícias.htm
pages/positivos/Receita de Bolo de abacaxi vegano.html
pages/positivos/tomate-recheado-com-proteina-de-soja-e-lentilha.html
pages/positivos/beijinho-vegano.htm

pages/positivos/maionese-de-alho.html
pages/positivos/barrinha-energetica-de-banana.html
pages/positivos/mini-hamburguer-de-feijao-branco-e-quinoa-com-molho-de-pepino.html
pages/positivos/hamburguer-vegano.html
pages/positivos/leite-de-castanha-do-para.html
pages/positivos/Baked Beans - gordelícias.htm
pages/positivos/salpicao-vegano.html
pages/positivos/salgado-de-hamburguer.html
pages/positivos/Receita de Repolho refogado com cenoura.html
pages/positivos/receita-de-refogado-de-brocolis-e-vagem-6922.html
pages/positivos/maionese-de-tofu.html
pages/positivos/fondue-de-chocolate-vegan.html
pages/positivos/tutu-de-feijao.html
pages/positivos/macarrao-com-brocolis-e-ervilhas.html
pages/positivos/salada-de-beterraba-assada.html
pages/positivos/ovo-de-pascoa-vegano.html
pages/positivos/molho-bolonhesa-de-lentilhas.html
pages/positivos/tortinha-de-morango-com-caramelo.html
pages/positivos/farofa-de-abobrinha.html
pages/positivos/estrogonofe-de-berinjela.html
pages/positivos/nhoque-vegano.htm

In [50]:
path = "pages/negativos"

parsedNegative = []

for filename in os.listdir(path):
    fullpath = os.path.join(path, filename)
    print(fullpath)
    if fullpath.endswith('.html'):
        page = soup(open(fullpath),'html.parser')
        table = page.find_all('p')
        s = "";
        pala_without_stop = "";
        p = [];
        for x in table:
                if(x.text != 'placeholder'):
                    s += x.text + " ";
        
        s=''.join(i for i in s if  not i.isdigit())
        for i in s.split():
            if i not in stopwords:
                pala_without_stop += i + " ";
        pala_without_stop = pala_without_stop.lower();
        pala_without_stop = pala_without_stop.translate(string.punctuation);
        parsed.append(pala_without_stop);
        y.append(0)

pages/negativos/Receita de Almondegas de Aveia.html
pages/negativos/Penne Funghi ao Forno.html
pages/negativos/Receita de Caldo de Sururu.html
pages/negativos/Receita de Escondidinho de bacalhau a minha moda.html
pages/negativos/receita-de-coxa-creme-passo-a-passo-6872.html
pages/negativos/chocotone-trufado-leite-ninho-e-doce-de-leite.html
pages/negativos/receita-de-ravioli-caseiro-de-carne-7042.html
pages/negativos/Receita de Panqueca de Whey Protein.html
pages/negativos/Receita de Crepioca - Pao de queijo de frigideira.html
pages/negativos/Receita de Creme de pitaya - Fácil.html
pages/negativos/receita-de-pizza-de-brocolis-low-carb-6850.html
pages/negativos/como-fazer-chantilly.html
pages/negativos/receita-de-bife-de-carne-moida-para-hamburguer-6989.html
pages/negativos/Receita de Cozido Russo.html
pages/negativos/receita-de-torta-de-sardinha-na-frigideira-7068.html
pages/negativos/receita-de-pizza-de-tapioca-na-frigideira-6988.html
pages/negativos/receita-de-esfiha-de-carne-moida-ab

pages/negativos/Panqueca de carne de sol.html
pages/negativos/pao-de-abobora-com-gotas-de-chocolate.html
pages/negativos/receita-de-asa-de-frango-com-molho-de-pimenta-5673.html
pages/negativos/Receita de Almondegas de tofu em molho de tomate.html
pages/negativos/Bife a Role.html
pages/negativos/Receita de Marisco vinagrete.html
pages/negativos/O negocio so nao foi fechado porque a Impossible Foods considerou a oferta.html
pages/negativos/receita-de-pudim-de-leite-condensado-sem-forno-5008.html
pages/negativos/Crespelle de Uva Itália do Chef Ravioli.html
pages/negativos/Churros de Cenoura.html
pages/negativos/receita-de-bolinho-de-arroz-integral-com-espinafre-6687.html
pages/negativos/como-fazer-kibe-frito-recheado.html
pages/negativos/receita-de-carne-de-panela-ao-molho-de-urucum-7053.html
pages/negativos/Receita de Bobo de camarao.html
pages/negativos/Receita de Sufle de queijo.html
pages/negativos/Receita de Panqueca de aveia solitária (pra uma pessoa so).html
pages/negativos/receita

# Feature Extraction and Classifier
<p>After getting the parsed text and put the tag on each test, we gonna use SkLearn to make the feature extraction with Bag of Words (CountVectorizer), transforming on frequencies with TF-IDF(TfidfTransformer) and using different classifiers. We also using the pipeline to reduce the steps and be more easy to do.</p>

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

## Used Classifiers

<p>SGClassifier -> Stochastic Gradient Descent</p>
<p>MultinomialNB -> Naive Bayes</p>
<p>DecisionTreeClassifier -> Arvore de Decisao</p>
<p>LogisticRegression -> Logistic Regression</p>
<p>MLPClassifier -> Multilayer Perceptron Classifier</p>
<p>SVC -> Support Vector Classification</p>

In [21]:
skf = StratifiedKFold(n_splits=15)
text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf2 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', SGDClassifier(loss='hinge',penalty='l1', max_iter=1000, tol=None))])
text_clf3 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', DecisionTreeClassifier())])
text_clf4 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', LogisticRegression())])
text_clf5 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MLPClassifier())])
text_clf6 = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', SVC())])


In [22]:
vectorizer = TfidfVectorizer()
vectorizer.fit(parsed)
print(vectorizer.vocabulary_)
print(vectorizer.idf_)

{'valoriza': 7017, 'precisar': 5370, 'costumava': 1738, 'estas': 2759, 'escumadeira': 2631, 'inspirar': 3762, 'cremes': 1795, 'esquecer': 2733, 'torrada': 6797, 'corte': 1724, 'retardar': 5980, 'stalliviere': 6433, 'versão': 7109, 'poupou': 5342, 'vieram': 7135, 'virava': 7152, 'inventado': 3808, 'famoso': 2914, 'potinho': 5333, 'pedra': 5052, 'caldart': 1036, 'aprovada': 495, 'maluca': 4208, 'lhe': 4048, 'descansou': 2055, 'prejudicasse': 5395, 'sensacional': 6247, 'mídias': 4600, 'garcia': 3314, 'mofarem': 4462, 'forminha': 3147, 'entendi': 2540, 'rampinha': 5727, 'dirá': 2241, 'erika': 2582, 'facilitaria': 2887, 'mascavo': 4295, 'bagunça': 721, 'encontrará': 2479, 'fervura': 3003, 'experimentar': 2852, 'preparações': 5410, 'casquinhas': 1156, 'senão': 6258, 'pelo': 5071, 'espremida': 2723, 'fotografia': 3168, 'emoji': 2434, 'desidratados': 2118, 'uvas': 7005, 'muitas': 4554, 'velocidade': 7072, 'igual': 3636, 'instrutor': 3769, 'definitivamente': 1954, 'experimentei': 2856, 'errar':

In [23]:
acuracia_tot = []
tempo_tot = []
precisao_tot = []
recall_tot = []

for i in range(1,16):
    print ('Epoch: ', i)
    X_train, X_test, y_train, y_test = train_test_split(parsed, y, test_size=0.3, random_state=None)
    classifiers = [text_clf, text_clf2, text_clf3, text_clf4, text_clf5, text_clf6]
    acuracia = []
    tempo = []
    precisao = []
    recall = []
    for classifier in classifiers:
        time1 = time.time()
        classifier.fit(X_train,y_train)
        time2 = time.time()
        y_pred = classifier.predict(X_test)
        score = classifier.score(X_test, y_test)
        precision = precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
        rec = recall_score(y_test, y_pred,  average='weighted', labels=np.unique(y_pred))
        acuracia.append(score)
        tempo.append(time2-time1)
        precisao.append(precision)
        recall.append(rec)
    acuracia_tot.append(acuracia)
    tempo_tot.append(tempo)
    precisao_tot.append(precisao)
    recall_tot.append(recall)

Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9
Epoch:  10
Epoch:  11
Epoch:  12
Epoch:  13
Epoch:  14
Epoch:  15


In [24]:
col = ['SGC', 'Naive Bayes', 'DecisionTree', 'LogisticRegression', 'MLP', 'SVC']
acuracia_pd = pd.DataFrame(acuracia_tot, columns = col)
acuracia_pd

,SGC,Naive Bayes,DecisionTree,LogisticRegression,MLP,SVC
0,0.794521,0.890411,0.812785,0.867580,0.863014,0.630137
1,0.876712,0.885845,0.831050,0.894977,0.908676,0.662100
2,0.831050,0.835616,0.794521,0.840183,0.844749,0.625571
3,0.831050,0.858447,0.799087,0.867580,0.849315,0.648402
4,0.867580,0.899543,0.867580,0.894977,0.885845,0.680365
5,0.776256,0.844749,0.794521,0.835616,0.849315,0.652968
6,0.789954,0.876712,0.808219,0.876712,0.872146,0.621005
7,0.808219,0.863014,0.808219,0.840183,0.853881,0.648402
8,0.872146,0.894977,0.899543,0.904110,0.876712,0.648402
9,0.826484,0.853881,0.812785,0.867580,0.863014,0.666667


In [25]:
print(np.mean(acuracia_pd))

SGC                   0.821309
Naive Bayes           0.868189
DecisionTree          0.821613
LogisticRegression    0.870624
MLP                   0.861796
SVC                   0.638965
dtype: float64


In [26]:
tempo_pd = pd.DataFrame(tempo_tot, columns = col)
tempo_pd

,SGC,Naive Bayes,DecisionTree,LogisticRegression,MLP,SVC
0,0.094893,0.542765,0.201446,0.198942,13.482364,0.382204
1,0.094026,0.548275,0.152350,0.096519,12.251887,0.397174
2,0.092048,0.544497,0.152241,0.098546,8.714294,0.390632
3,0.093577,0.552904,0.143222,0.098515,10.743105,0.401891
4,0.090600,0.540673,0.147567,0.096930,11.301198,0.395801
5,0.092844,0.531946,0.145679,0.098604,8.932450,0.396730
6,0.091840,0.564950,0.140121,0.097767,9.236183,0.382645
7,0.092303,0.543114,0.154044,0.098163,10.322619,0.395199
8,0.090655,0.538049,0.148590,0.096314,9.464125,0.387349
9,0.092382,0.517237,0.139164,0.097634,11.357982,0.398428


In [27]:
print(np.mean(tempo_pd))

SGC                    0.092445
Naive Bayes            0.545920
DecisionTree           0.149905
LogisticRegression     0.104469
MLP                   10.123259
SVC                    0.389489
dtype: float64


In [28]:
precisao_pd = pd.DataFrame(precisao_tot, columns = col)
precisao_pd

,SGC,Naive Bayes,DecisionTree,LogisticRegression,MLP,SVC
0,0.818744,0.889987,0.811586,0.866796,0.862439,0.630137
1,0.888222,0.884819,0.830519,0.895359,0.909423,0.662100
2,0.832718,0.844346,0.792610,0.845944,0.848048,0.625571
3,0.832676,0.857462,0.803511,0.867218,0.849792,0.648402
4,0.876752,0.901618,0.874528,0.901246,0.887440,0.680365
5,0.771358,0.850364,0.814649,0.856313,0.855788,0.652968
6,0.821836,0.876083,0.807420,0.876455,0.871459,0.621005
7,0.809556,0.863950,0.835667,0.841859,0.859205,0.648402
8,0.888898,0.894354,0.899087,0.903860,0.875908,0.648402
9,0.828665,0.859724,0.818465,0.870613,0.864074,0.666667


In [29]:
print(np.mean(precisao_pd))

SGC                   0.834328
Naive Bayes           0.869466
DecisionTree          0.826098
LogisticRegression    0.872847
MLP                   0.862996
SVC                   0.638965
dtype: float64


In [30]:
recall_pd = pd.DataFrame(recall_tot, columns = col)
recall_pd

,SGC,Naive Bayes,DecisionTree,LogisticRegression,MLP,SVC
0,0.794521,0.890411,0.812785,0.867580,0.863014,1.0
1,0.876712,0.885845,0.831050,0.894977,0.908676,1.0
2,0.831050,0.835616,0.794521,0.840183,0.844749,1.0
3,0.831050,0.858447,0.799087,0.867580,0.849315,1.0
4,0.867580,0.899543,0.867580,0.894977,0.885845,1.0
5,0.776256,0.844749,0.794521,0.835616,0.849315,1.0
6,0.789954,0.876712,0.808219,0.876712,0.872146,1.0
7,0.808219,0.863014,0.808219,0.840183,0.853881,1.0
8,0.872146,0.894977,0.899543,0.904110,0.876712,1.0
9,0.826484,0.853881,0.812785,0.867580,0.863014,1.0


In [31]:
print(np.mean(recall_pd))

SGC                   0.821309
Naive Bayes           0.868189
DecisionTree          0.821613
LogisticRegression    0.870624
MLP                   0.861796
SVC                   1.000000
dtype: float64


# Feature Selection

Testando selecionar as features previamentes para verificar se houve melhora no classificador


In [32]:
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_extraction.text import CountVectorizer

In [34]:
cv = CountVectorizer(max_df=0.9, max_features=1000)
X_vec = cv.fit_transform(X_train)
res = dict(zip(mutual_info_classif(X_vec, y_train, discrete_features=True),cv.get_feature_names()
               ))
res

{5.437201279906434e-07: 'internet',
 6.4903492988099831e-06: 'principal',
 9.197442031289027e-06: 'consistência',
 1.4814927698475477e-05: 'bater',
 6.3011258457638941e-05: 'rápido',
 6.8436176272321236e-05: 'vão',
 7.197312972541392e-05: 'fáceis',
 0.00010224773954006436: 'forma',
 0.00010976295709561046: 'mesma',
 0.00016675110321931474: 'alumínio',
 0.00020421771450516643: 'frite',
 0.00020471344146480094: 'disso',
 0.00021246703776998943: 'refeições',
 0.00022611113129540443: 'espalhe',
 0.00023749966628218953: 'distribua',
 0.00025672923099991432: 'forminhas',
 0.00026775085312308496: 'gelada',
 0.00028170881886039473: 'recipiente',
 0.00031094334824682823: 'contato',
 0.00034643576575234258: 'saborosa',
 0.00035078752273540875: 'restantes',
 0.00036159722754683377: 'boa',
 0.00036258716639851231: 'pasta',
 0.00039482218502644162: 'mão',
 0.00040930147892816876: 'iguais',
 0.00041123508201895004: 'crocante',
 0.00042262535545745106: 'precisa',
 0.00045662529685844624: 'perfeito',


In [35]:
text_clf = Pipeline([('vect', cv), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf2 = Pipeline([('vect', cv), ('tfidf', TfidfTransformer()), ('clf', SGDClassifier(loss='hinge',penalty='l1', max_iter=1000, tol=None))])
text_clf3 = Pipeline([('vect', cv), ('tfidf', TfidfTransformer()), ('clf', DecisionTreeClassifier())])
text_clf4 = Pipeline([('vect', cv), ('tfidf', TfidfTransformer()), ('clf', LogisticRegression())])
text_clf5 = Pipeline([('vect', cv), ('tfidf', TfidfTransformer()), ('clf', MLPClassifier())])
text_clf6 = Pipeline([('vect', cv), ('tfidf', TfidfTransformer()), ('clf', SVC())])

In [36]:
acuracia_tot2 = []
tempo_tot2 = []
precisao_tot2 = []
recall_tot2 = []

for i in range(1,16):
    print ('Epoch: ', i)
    X_train, X_test, y_train, y_test = train_test_split(parsed, y, test_size=0.3, random_state=None)
    classifiers = [text_clf, text_clf2, text_clf3, text_clf4, text_clf5, text_clf6]
    acuracia = []
    tempo = []
    precisao = []
    recall = []
    for classifier in classifiers:
        time1 = time.time()
        classifier.fit(X_train,y_train)
        time2 = time.time()
        y_pred = classifier.predict(X_test)
        score = classifier.score(X_test, y_test)
        precision = precision_score(y_test, y_pred, average='weighted', labels=np.unique(y_pred))
        rec = recall_score(y_test, y_pred,  average='weighted', labels=np.unique(y_pred))
        acuracia.append(score)
        tempo.append(time2-time1)
        precisao.append(precision)
        recall.append(rec)
    acuracia_tot2.append(acuracia)
    tempo_tot2.append(tempo)
    precisao_tot2.append(precisao)
    recall_tot2.append(recall)

Epoch:  1
Epoch:  2
Epoch:  3
Epoch:  4
Epoch:  5
Epoch:  6
Epoch:  7
Epoch:  8
Epoch:  9
Epoch:  10
Epoch:  11
Epoch:  12
Epoch:  13
Epoch:  14
Epoch:  15


In [37]:
col = ['SGC', 'Naive Bayes', 'DecisionTree', 'LogisticRegression', 'MLP', 'SVC']
acuracia_pd2 = pd.DataFrame(acuracia_tot2, columns = col)
acuracia_pd2

,SGC,Naive Bayes,DecisionTree,LogisticRegression,MLP,SVC
0,0.863014,0.890411,0.812785,0.917808,0.885845,0.611872
1,0.840183,0.821918,0.844749,0.863014,0.840183,0.657534
2,0.831050,0.890411,0.831050,0.894977,0.890411,0.648402
3,0.812785,0.858447,0.771689,0.881279,0.853881,0.657534
4,0.840183,0.863014,0.853881,0.872146,0.858447,0.643836
5,0.799087,0.853881,0.835616,0.853881,0.844749,0.634703
6,0.844749,0.876712,0.858447,0.881279,0.867580,0.643836
7,0.849315,0.894977,0.863014,0.876712,0.863014,0.657534
8,0.872146,0.894977,0.858447,0.913242,0.876712,0.643836
9,0.867580,0.858447,0.890411,0.913242,0.853881,0.662100


In [38]:
print(np.mean(acuracia_pd2))

SGC                   0.834703
Naive Bayes           0.870015
DecisionTree          0.833181
LogisticRegression    0.880974
MLP                   0.860274
SVC                   0.639269
dtype: float64


In [39]:
tempo_pd2 = pd.DataFrame(tempo_tot2, columns = col)
tempo_pd2

,SGC,Naive Bayes,DecisionTree,LogisticRegression,MLP,SVC
0,0.096458,0.512476,0.126307,0.098169,4.061267,0.334238
1,0.096793,0.505252,0.124947,0.098995,3.310672,0.346060
2,0.099341,0.526110,0.125007,0.099746,4.457618,0.345939
3,0.094470,0.557616,0.131083,0.158902,3.487830,0.340724
4,0.092475,0.520195,0.126638,0.096252,3.495382,0.332408
5,0.092805,0.498020,0.125934,0.095300,3.653046,0.326463
6,0.092246,0.489239,0.128328,0.095638,3.431278,0.329700
7,0.091394,0.512852,0.131460,0.096022,3.863690,0.329694
8,0.092568,0.537947,0.116239,0.095439,3.988198,0.325605
9,0.092921,0.494511,0.128912,0.099497,3.592637,0.333977


In [40]:
print(np.mean(tempo_pd2))

SGC                   0.093619
Naive Bayes           0.511642
DecisionTree          0.125814
LogisticRegression    0.100723
MLP                   3.659924
SVC                   0.332388
dtype: float64


In [41]:
precisao_pd2 = pd.DataFrame(precisao_tot2, columns = col)
print(precisao_pd2)
print(np.mean(precisao_pd2))

         SGC  Naive Bayes  DecisionTree  LogisticRegression       MLP  \
0   0.883836     0.890411      0.811260            0.920137  0.886874   
1   0.859550     0.825404      0.843873            0.864011  0.839700   
2   0.863625     0.889660      0.842017            0.895323  0.890411   
3   0.842782     0.864884      0.771689            0.883311  0.853881   
4   0.846527     0.863014      0.856024            0.872146  0.859933   
5   0.836367     0.853226      0.837800            0.857256  0.843571   
6   0.858919     0.878058      0.858883            0.883153  0.867990   
7   0.871607     0.898958      0.866720            0.889508  0.865248   
8   0.888497     0.895312      0.857508            0.913526  0.875947   
9   0.883561     0.857320      0.891271            0.913565  0.856283   
10  0.836762     0.899775      0.794309            0.894945  0.849597   
11  0.854863     0.860221      0.851745            0.887579  0.864107   
12  0.843234     0.899543      0.871409            

In [42]:
recall_pd2 = pd.DataFrame(recall_tot2, columns = col)
print(recall_pd2)
print(np.mean(recall_pd2))

         SGC  Naive Bayes  DecisionTree  LogisticRegression       MLP  SVC
0   0.863014     0.890411      0.812785            0.917808  0.885845  1.0
1   0.840183     0.821918      0.844749            0.863014  0.840183  1.0
2   0.831050     0.890411      0.831050            0.894977  0.890411  1.0
3   0.812785     0.858447      0.771689            0.881279  0.853881  1.0
4   0.840183     0.863014      0.853881            0.872146  0.858447  1.0
5   0.799087     0.853881      0.835616            0.853881  0.844749  1.0
6   0.844749     0.876712      0.858447            0.881279  0.867580  1.0
7   0.849315     0.894977      0.863014            0.876712  0.863014  1.0
8   0.872146     0.894977      0.858447            0.913242  0.876712  1.0
9   0.867580     0.858447      0.890411            0.913242  0.853881  1.0
10  0.812785     0.899543      0.794521            0.894977  0.849315  1.0
11  0.831050     0.858447      0.853881            0.876712  0.863014  1.0
12  0.831050     0.899543

In [43]:
print(np.mean(acuracia_pd2) - np.mean(acuracia_pd))

SGC                   0.013394
Naive Bayes           0.001826
DecisionTree          0.011568
LogisticRegression    0.010350
MLP                  -0.001522
SVC                   0.000304
dtype: float64


In [45]:
from collections import Counter

new_list = []
for i in parsed:
    new_list += i.split()
Counter = Counter(new_list)
most_occur = Counter.most_common(100)
print(most_occur)

[('(de', 1951), ('é', 1685), ('chá)', 1402), ('receita', 1379), ('xícara', 1221), ('coloque', 1190), ('água', 841), ('bem', 819), ('receitas', 794), ('deixe', 720), ('sal', 712), ('principalmente', 660), ('adicione', 654), ('colher', 633), ('leite', 619), ('ingredientes', 602), ('gosto', 579), ('se', 577), ('em', 577), ('colheres', 558), ('sopa)', 553), ('misture', 521), ('pode', 508), ('massa', 502), ('óleo', 477), ('aqui', 477), ('fazer', 474), ('cerca', 468), ('acrescente', 462), ('molho', 455), ('farinha', 450), ('ficar', 443), ('fogo', 432), ('alho', 419), ('cebola', 410), ('panela', 406), ('forno', 404), ('azeite', 402), ('minutos', 378), ('frango', 368), ('leve', 361), ('culinária', 361), ('apaixonada', 357), ('sobre', 354), ('desde', 352), ('minutos.', 346), ('açúcar', 340), ('pratos', 337), ('muitos', 336), ('vegetariana', 333), ('testar', 333), ('cozinhar,', 331), ('designer', 321), ('pequena.', 321), ('aprendeu', 320), ('conhece', 319), ('fotógrafa', 319), ('animais', 319), 